In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
!which ipython

/home/prise6/anaconda3/envs/py35/bin/ipython


In [4]:
import logging
from pathlib import Path
from src.data.WordData import WordData
import json
import numpy as np
from src.models.WordDataLoader import WordDataLoader
from src.models.simpleLSTMModel import simpleLSTMModel
from src.models.WordDataLoader import IdWordDataLoader
from src.models.ModelTrainer import ModelTrainer
from src.models.TweetGenerator import TweetGenerator
from src.models.GenerateTweetCallback import GenerateTweetCallback
from keras.models import load_model

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
NUM_STEP = 20
BATCH_SIZE = 256
NB_TOKEN_KEEP = 0
NB_EPOCH = 1
LSTM_HIDDEN_SIZE = 100

In [6]:
with open('../data/interim/tokens.json') as f:
    tokens = json.load(f)

with open('../data/interim/tokens_id_keep.json') as f:
    tokens_id_keep = json.load(f)

word_data = WordData(tokens, tokens_id_keep, NB_TOKEN_KEEP)

# print(word_data.token_final[1:4])

print(word_data.getVocabularyLength())

# print(word_data.ref_word_to_id)

sentence_loader = IdWordDataLoader(20, 5, word_data, 1, 1)

# for x,y in word_data_loader.generate():
# 	print(x)
# 	print(y)
# 	break

lstm_model = load_model('../models/simpleLSTMModel/version2/final_model_colab.hdf5')
# lstm_model = simpleLSTMModel(word_data.getVocabularyLength(), LSTM_HIDDEN_SIZE, (None, None))
# print(lstm_model.model.summary())

# lstm_model.save()
# callbacks = [generate_tweet_callback]
# trainer = ModelTrainer(lstm_model, word_data_loader, NB_EPOCH, workers = 3, use_multiprocessing = True, callbacks = callbacks)
# trainer.train()

# lstm_model.save_directory = '../models/simpleLSTMModel/version1'
# lstm_model.load('model-60')
print(lstm_model.model.summary())

31418
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 500)         15709000  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 500)         2002000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 500)         2002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 500)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 31418)       15740418  
_________________________________________________________________
activation_1 (Activation)    (None, None, 31418)       0         
Total params: 35,453,418
Trainable params: 35,453,418
Non-trainable params: 0
__________________________________________________________

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [10]:
def generateWithModel(lstm_model, phrases, word_data, seed = 8736, stop = 144):

    tweet_generator = TweetGenerator(lstm_model, word_data)
    
    res = []
    
    for phrase in phrases:
        id_token = [word_data.ref_word_to_id[word_data.sentence_token_start]]
        for mot in phrase:
            id_token.append(word_data.ref_word_to_id[mot])
        res.append(' '.join(tweet_generator.tweet(np.array(id_token), stop = stop, seed = seed)))
        
    return res

In [7]:
lstm_model_20 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-20"))
lstm_model_40 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-40"))
lstm_model_60 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-60"))
lstm_model_80 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-80"))
lstm_model_100 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-100"))
# lstm_model_140 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-140"))
# lstm_model_200 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-200"))
#lstm_model_260 = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("model-260"))


In [16]:
lstm_model_final = load_model('../models/simpleLSTMModel/version2/{}.hdf5'.format("final_model_colab"))

In [8]:
phrases = [
    [],
    ['pas', 'content', 'de'],
    ['c\'', 'est', 'bien'], 
    ['je', 'veux', 'trouver'],
    ['les', 'jeux', 'sont'],
    ['c\'', 'est', 'toujours'],
    ['il', 'est']
]

In [11]:
generateWithModel(lstm_model_20, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


-0.3448953


['_SENTENCE_START_ _MENTION_ _MENTION_ _MENTION_ _MENTION_ _MENTION_ _MENTION_ _MENTION_ _MENTION_ _HASHTAG_ _HASHTAG_ _HASHTAG_ _HASHTAG_ _HASHTAG_ _SENTENCE_STOP_',
 '_SENTENCE_START_ pas content de _HASHTAG_ pour les #jo2024 ! _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est bien que la candidature de la candidature de #paris2024 _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver la candidature de la candidature de #paris2024 ! _SENTENCE_STOP_',
 '_SENTENCE_START_ les jeux sont un _HASHTAG_ pour les #jo2024 _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours que pour #paris2024 ! _SENTENCE_STOP_",
 "_SENTENCE_START_ il est pas pas que les #jo2024 , c' est bien en _HASHTAG_ ! _SENTENCE_STOP_"]

In [12]:
generateWithModel(lstm_model_40, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


['_SENTENCE_START_ _MENTION_ _MENTION_ rigoureusement contre les #jo2024 a #paris ! _SENTENCE_STOP_',
 '_SENTENCE_START_ pas content de rio pour defendre #paris2024 ! _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est bien pour la candidature de #paris2024 ! _HASHTAG_ _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver la twirling du monde des _HASHTAG_ ? _HASHTAG_ ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?',
 '_SENTENCE_START_ les jeux sont tous les jeux olympiques de 2024 ? #paris2024 _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours que pour enrichir la candidature de #paris2024 ! _SENTENCE_STOP_",
 '_SENTENCE_START_ il est pas mauvais pour les #jo2024 a #paris ! _SENTENCE_STOP_']

In [13]:
generateWithModel(lstm_model_60, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


-0.3936073
-0.34988877


['_SENTENCE_START_ _MENTION_ _MENTION_ ils sont prets pour le pogrom , heu .. pardon le podium #paris2024 _SENTENCE_STOP_',
 "_SENTENCE_START_ pas content de l' hotel de ville de paris pour feter les _HASHTAG_ _SENTENCE_STOP_",
 "_SENTENCE_START_ c' est bien connu . _HASHTAG_ . _MENTION_ . hate de voir le _HASHTAG_ anime par herve , marie de canal+ et _MENTION_ #paris2024 _HASHTAG_ _MENTION_ _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver la couleur de la tour eiffel . _SENTENCE_STOP_',
 '_SENTENCE_START_ les jeux sont les artistes , et quoi . #paris2024 _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours pour les _HASHTAG_ ! bonne programmation je trouve ! _HASHTAG_ _SENTENCE_STOP_",
 '_SENTENCE_START_ il est venu juste pour faire gueuler les meufs avec sa musique pourri la et il est reparti . #paris2024 _SENTENCE_STOP_']

In [14]:
generateWithModel(lstm_model_80, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


-0.3004694
-0.38103834
-0.30244902
-0.32534638
-0.35464707
-0.34399423
-0.30064476
-0.42015317


['_SENTENCE_START_ _MENTION_ _MENTION_ _MENTION_ @lafncc @univavignon _MENTION_ _MENTION_ _MENTION_ beau chantier pour prepa #jo2024 @ilfautallervoir . _SENTENCE_STOP_',
 '_SENTENCE_START_ pas content de vous faire oublier les francais . tout ca pour les _HASHTAG_ ? _MENTION_ _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est bien connu . . ! - une chance pour les _HASHTAG_ _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver la champagne pour #paris2024 ? ! ? #paris2024 _SENTENCE_STOP_',
 '_SENTENCE_START_ les jeux sont la fete du sport du _HASHTAG_ ! avec les athletes pour la candidature de #paris2024 ... avec _MENTION_ pour la _HASHTAG_ . quel concert _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours pour un grand challenge olympique pour les _HASHTAG_ _SENTENCE_STOP_",
 '_SENTENCE_START_ il est etre laura avec une victoire de #paris2024 _SENTENCE_STOP_']

In [15]:
generateWithModel(lstm_model_100, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


-0.33798918
-0.39059675
-0.3679066
-0.31954038
-0.37466836


['_SENTENCE_START_ _MENTION_ _MENTION_ _MENTION_ @lafncc @univavignon _MENTION_ _MENTION_ _MENTION_ beau chantier pour prepa #jo2024 @ilfautallervoir . _SENTENCE_STOP_',
 "_SENTENCE_START_ pas content de l' impact des jeux olympiques de sarajevo 84 #paris2024 _SENTENCE_STOP_",
 "_SENTENCE_START_ c' est bien mignon de vouloir organiser les jo d' ete 2024 en france mais bon le minimum ca serait d' avoir un ete kua #jo2024 _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver #paris2024 pour #paris2024 _SENTENCE_STOP_',
 '_SENTENCE_START_ les jeux sont une source de bonheur pour _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours l' ? _HASHTAG_ _HASHTAG_ #paris2024 _SENTENCE_STOP_",
 "_SENTENCE_START_ il est plus que la france est a l' elysee avec la candidature aux jo de #paris2024 _SENTENCE_STOP_"]

In [17]:
generateWithModel(lstm_model_final, phrases, word_data)

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


-0.4298332
-0.33117563
-0.32590643


['_SENTENCE_START_ _MENTION_ _MENTION_ rigoureusement contre les #jo2024 a #paris ! _SENTENCE_STOP_',
 "_SENTENCE_START_ pas content de l' impact des jeux olympiques de sarajevo 84 #paris2024 _SENTENCE_STOP_",
 "_SENTENCE_START_ c' est bien que _HASHTAG_ en personne aille defendre le projet #paris2024 a _HASHTAG_ . ca garantit l' echec de ce projet grotesque . _HASHTAG_ _SENTENCE_STOP_",
 '_SENTENCE_START_ je veux trouver la couleur de la tour eiffel . _SENTENCE_STOP_',
 '_SENTENCE_START_ les jeux sont les retransmissions se feraient via internet et non pas via la tv.lire // _SENTENCE_STOP_',
 "_SENTENCE_START_ c' est toujours quand je suis pour ms la les _HASHTAG_ vont comme pour la france ! _SENTENCE_STOP_",
 '_SENTENCE_START_ il est bien parti pour la photo de #paris avec _MENTION_ _SENTENCE_STOP_']